In [14]:
#importing Necessary Libraries
import requests #To make HTTP requests
from bs4 import BeautifulSoup # To parse HTML content
import pandas as pd # For data manipulation and storage
import time # To create delays between requests
import os # For interacting with the operating system 
from requests.adapters import HTTPAdapter # To handle retries for HTTP requests
from requests.packages.urllib3.util.retry import Retry # To define retry logic


In [15]:
# Checkpoint file names
PRODUCT_URLS_FILE = 'fashion_urls_checkpoint.csv' # File to store the checkpoint of scraped product URLs
REVIEWS_FILE = 'fashion_reviews_checkpoint.csv' # File to store the checkpoint of scraped reviews

In [16]:
# Function to create a session with retry logic
# Create a session object to persist certain parameters across requests and 
# define retry strategy to handle transient errors
    session = requests.Session()
def create_session():
    session = requests.Session()
    retry = Retry(
        total=5,
        read=5,
        connect=5,
        backoff_factor=0.3,
        status_forcelist=(500, 502, 503, 504),
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    # Mount the adapter with retry logic to the session
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

In [17]:
# Function to get product URLs from a single listing page
def get_product_urls(listing_url):
    session = create_session() # Create a session with retry logic
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    # Attempt to send a request to the listing page
    try:
        response = session.get(listing_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []
    
    # Parse the HTML content of the page    
    soup = BeautifulSoup(response.content, 'html.parser')
    product_urls = []
    
    # Find all product links on the page
    for a_tag in soup.find_all('a', class_='core'):
        url = a_tag.get('href')
        if url:
            product_urls.append('https://www.jumia.com.ng' + url)
            
    return product_urls

In [18]:
# Function to get the maximum number of review pages for a product
def get_max_review_pages(product_url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
     # Attempt to send a request to the product page
    try:
        response = session.get(product_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return 1

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Determine the maximum number of review pages
    pagination = soup.find('div', class_='pg-w -ptm -pbxl')
    if pagination:
        pages = pagination.find_all('a')
        page_numbers = [int(page.text.strip()) for page in pages if page.text.strip().isdigit()]
        if page_numbers:
            return max(page_numbers)
    return 1  # Default to 1 if pagination is not found


In [19]:
# Function to get product reviews from a single page
def get_product_reviews(url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    reviews = []
    review_elements = soup.find_all('article', class_='-pvs -hr _bet')

    # Extract review details
    for element in review_elements:
        review = {}
        
        rating_tag = element.find('div', class_='stars _m _al -mvs')
        review['rating'] = rating_tag.text.split(' out of ')[0].strip() if rating_tag else 'N/A'
        
        title_tag = element.find('h3', class_='-m -fs16 -pvs')
        review['title'] = title_tag.text.strip() if title_tag else 'N/A'
        
        body_tag = element.find('p', class_='-pvs')
        review['body'] = body_tag.text.strip() if body_tag else 'N/A'
        
        date_tag = element.find('span', class_='-prs')
        review['date'] = date_tag.text.strip() if date_tag else 'N/A'
        
        name_tag = element.find('span', string=lambda x: x and x.startswith('by '))
        review['name'] = name_tag.text.replace('by ', '').strip() if name_tag else 'N/A'
        
        reviews.append(review)

    return reviews

In [20]:
# Function to get all reviews for a given product URL
def get_all_reviews(product_url):
    max_pages = get_max_review_pages(product_url) # Determine the maximum number of review pages
    all_reviews = []

    # Loop through all review pages and collect reviews
    for page in range(1, max_pages + 1):
        url = f"{product_url}?page={page}"
        reviews = get_product_reviews(url)
        all_reviews.extend(reviews)
        time.sleep(0.5)  # Add a delay to avoid hitting the server too quickly
    
    return all_reviews # Return the list of all reviews

In [21]:
# Function to save product URLs to a CSV file (checkpoint)
def save_product_urls_checkpoint(product_urls):
    df = pd.DataFrame(product_urls, columns=['product_url'])
    df.to_csv(PRODUCT_URLS_FILE, index=False)

In [22]:
# Function to load product URLs from a CSV file (checkpoint)
def load_product_urls_checkpoint():
    if os.path.exists(PRODUCT_URLS_FILE):
        df = pd.read_csv(PRODUCT_URLS_FILE)
        return df['product_url'].tolist()
    return []

In [23]:
# Function to save reviews to a CSV file (checkpoint)
def save_reviews_checkpoint(reviews):
    df = pd.DataFrame(reviews)
    if os.path.exists(REVIEWS_FILE):
        df.to_csv(REVIEWS_FILE, mode='a', header=False, index=False)
    else:
        df.to_csv(REVIEWS_FILE, index=False)

In [24]:
# Function to load reviews from a CSV file (checkpoint)
def load_reviews_checkpoint():
    if os.path.exists(REVIEWS_FILE):
        return pd.read_csv(REVIEWS_FILE).to_dict('records')
    return []

In [25]:
# Function to scrape multiple products
def scrape_multiple_products(product_urls):
    all_product_reviews = load_reviews_checkpoint() # Load previously scraped reviews
    
    # Get already scraped product URLs to avoid reprocessing
    scraped_product_urls = {review['product_url'] for review in all_product_reviews}
    
    # Loop through all product URLs to scrape reviews
    for product_url in product_urls:
        if product_url not in scraped_product_urls: # Skip already processed URLs
            print(f"Scraping reviews for: {product_url}")
            product_reviews = get_all_reviews(product_url) # Scrape all reviews for the product
            for review in product_reviews:
                review['product_url'] = product_url  # Add product URL to each review for checkpointing
            all_product_reviews.extend(product_reviews)
            save_reviews_checkpoint(product_reviews)  # Save progress
            scraped_product_urls.add(product_url)
    
    return all_product_reviews

In [26]:
def main():
    base_listing_url = 'https://www.jumia.com.ng/category-fashion-by-jumia/?page='  # Base URL for the listing pages
    total_listing_pages = 50  # Total number of listing pages to scrape

    # Load previously scraped product URLs to avoid reprocessing
    all_product_urls = load_product_urls_checkpoint()
    last_page_scraped = len(all_product_urls) // 40  # Assuming 40 products per page

    # Step 1: Get product URLs from all listing pages
    for page in range(last_page_scraped + 1, total_listing_pages + 1):
        listing_url = f"{base_listing_url}{page}"
        product_urls = get_product_urls(listing_url)
        all_product_urls.extend(product_urls)
        print(f"Page {page}: Found {len(product_urls)} product URLs.")
        save_product_urls_checkpoint(all_product_urls)  # Save progress
    
    print(f"Total product URLs found: {len(all_product_urls)}")

    # Step 2: Scrape reviews for each product URL
    all_reviews = scrape_multiple_products(all_product_urls)

    if all_reviews:
        df = pd.DataFrame(all_reviews)
        # Reorder the columns for better readability
        df = df[['name', 'rating', 'date', 'title', 'body', 'product_url']]
        print(df)
        # Save dataframe to a CSV file
        filename = os.path.join(os.getcwd(), 'jumia_NLP_fashiondataset.csv')
        df.to_csv(filename, index=False)
        print(f"File saved as {filename}")

        # Additional confirmation
        if os.path.exists(filename):
            print(f"Confirmed: {filename} has been saved.")
        else:
            print(f"Error: {filename} was not saved.")
    else:
        print("No reviews found.")

if __name__ == '__main__':
    main()

Total product URLs found: 2095
Scraping reviews for: https://www.jumia.com.ng/osc-signature-mens-casual-100-cotton-t-shirt-micro-dot-351576262.html
Scraping reviews for: https://www.jumia.com.ng/osc-signature-mens-quality-100-cotton-single-round-neck-t-shirt-with-aircutdesign-350683110.html
Scraping reviews for: https://www.jumia.com.ng/osc-signature-men-premium-100-cotton-5-in-1-t-shirt-bundle-350972168.html
Scraping reviews for: https://www.jumia.com.ng/osc-signature-mens-quality-100-cotton-single-t-shirt-with-hustle-and-grind-design-350685716.html
Scraping reviews for: https://www.jumia.com.ng/adidas-glideease-w-women-274940967.html
Scraping reviews for: https://www.jumia.com.ng/adidas-altaventure-sport-swim-sandals-unisex-272266368.html
Scraping reviews for: https://www.jumia.com.ng/fashion-t-shirts-shorts-100-cotton-mens-short-sleeve-set-white-94668946.html
Scraping reviews for: https://www.jumia.com.ng/fashion-cuban-link-chain-with-cross-pendant-silver-209921044.html
Scraping rev

Scraping reviews for: https://www.jumia.com.ng/fashion-original-cuban-chain-gold-119929939.html
Scraping reviews for: https://www.jumia.com.ng/fashion-2in1-super-facinating-full-studded-cuban-chain-with-classic-iced-pendant-for-rugged-men-141958776.html
Scraping reviews for: https://www.jumia.com.ng/legend-premiums-all-black-mens-complete-jewelry-set-matching-wristwatch-for-black-legends-142732481.html
Scraping reviews for: https://www.jumia.com.ng/fashion-zirconia-ice-luxury-shiny-earrings-135459386.html
Scraping reviews for: https://www.jumia.com.ng/luxury-collection-top-quality-rugged-30mm-gold-miami-cuban-link-bracelet-for-bigger-boys-214440578.html
Scraping reviews for: https://www.jumia.com.ng/fashion-black-brown-double-sided-reversible-men-leather-belt-122289990.html
Scraping reviews for: https://www.jumia.com.ng/fashion-men-studded-iced-wristwatch-gold-with-cuban-link-chain-284021829.html
Scraping reviews for: https://www.jumia.com.ng/keep-moving-first-class-powerful-non-fading

Scraping reviews for: https://www.jumia.com.ng/fashion-4-piece-multifunctional-shoulder-bag-black-230667777.html
Scraping reviews for: https://www.jumia.com.ng/fashion-men-classy-silicon-watch-91043506.html
Scraping reviews for: https://www.jumia.com.ng/fashion-3-in-1-set-of-fitted-unisex-long-sleeve-shirt.-107010734.html
Scraping reviews for: https://www.jumia.com.ng/fashion-british-casual-shoe-man-black-93087100.html
Scraping reviews for: https://www.jumia.com.ng/fashion-business-dress-shoes-for-young-men-england-black-round-head-leisure-shoes-107398236.html
Scraping reviews for: https://www.jumia.com.ng/pin-time-luxurious-deluxe-magnificent-set-of-top-wristwatch-iced-necklace-bracelet-for-luxury-menwomen-141966306.html
Scraping reviews for: https://www.jumia.com.ng/fashion-dinner-gown-for-ladies-women-office-skirt-casual-turkey-wears-dress-38386273.html
Scraping reviews for: https://www.jumia.com.ng/fashion-high-top-sneaker-soft-mesh-breathable-for-men-footwear-shoes-black-white-268

Scraping reviews for: https://www.jumia.com.ng/fashion-broadstreet-hybrid-trainers-grey-107234429.html
Scraping reviews for: https://www.jumia.com.ng/fashion-iced-out-cuban-chain-and-clown-pendant-silver-85709079.html
Scraping reviews for: https://www.jumia.com.ng/fashion-the-lion-king-3d-print-mens-zipper-sweatshirt-set-258906165.html
Scraping reviews for: https://www.jumia.com.ng/generic-stylish-vintage-male-female-sunglass-uv-400-square-sun-glass-257154421.html
Scraping reviews for: https://www.jumia.com.ng/fashion-cat-eye-anti-blue-light-prescriptionable-optical-eyeglasses-242492360.html
Scraping reviews for: https://www.jumia.com.ng/fashion-unisex-stylish-fashion-leather-beret-black-120982329.html
Scraping reviews for: https://www.jumia.com.ng/yemlays-sexy-stretch-cuffs-womens-shredded-slim-shorts-jeans-blue-136136980.html
Scraping reviews for: https://www.jumia.com.ng/fashion-nose-ring-reusable-non-piercing-alloy-fake-magnetic-horseshoe-for-party-silver-82801717.html
Scraping rev

Scraping reviews for: https://www.jumia.com.ng/fashion-zirconia-ice-luxury-shiny-earrings-137170867.html
Scraping reviews for: https://www.jumia.com.ng/fashion-two-in-one-classic-suit-trousers-for-men-blackgrey-color-110881919.html
Scraping reviews for: https://www.jumia.com.ng/fashion-brown-leather-sandals-for-men-194474368.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-elegant-costume-jewelry-set-silver-color-89061941.html
Scraping reviews for: https://www.jumia.com.ng/festina-sophisticated-non-fading-mens-gold-wristwatchnecklacehand-chainstud-earrings-216615686.html
Scraping reviews for: https://www.jumia.com.ng/brazillian-gold-long-lasting-original-quality-non-fading-couples-wedding-ring-143357020.html
Scraping reviews for: https://www.jumia.com.ng/fashion-iced-bold-men-non-fade-high-quality-cuban-chain-silver-144178486.html
Scraping reviews for: https://www.jumia.com.ng/fashion-2-in-1-unisex-crystal-rhinestone-led-fashion-quartz-wristwatch-83746148.html
Scrapin

...

Scraping reviews for: https://www.jumia.com.ng/fashion-romantic-her-beast-king-his-beauty-queen-couple-lovers-titanium-steel-bracelet-black-her-king-77181946.html
Scraping reviews for: https://www.jumia.com.ng/fashion-men-classy-chain-brooch-81189011.html
Scraping reviews for: https://www.jumia.com.ng/dynamite-womens-embroidered-floral-loafers-canvas-slip-on-shoes-for-women-pink-267601050.html
Scraping reviews for: https://www.jumia.com.ng/fashion-womens-square-shape-elegant-small-leather-strap-ladies-wrist-watch-girls-watch-221806532.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-wristwatch-wallet-face-cap-for-men-116452133.html
Scraping reviews for: https://www.jumia.com.ng/fashion-baby-girls-dressprincess-kids-dressbaby-girl-ball-dress-347770304.html
Scraping reviews for: https://www.jumia.com.ng/keep-moving-super-exotic-classy-non-fading-ladies-gold-wristwatch-bracelet-for-queens-82874577.html
Scraping reviews for: https://www.jumia.com.ng/fashion-silver-iced-out-

Scraping reviews for: https://www.jumia.com.ng/generic-anti-blue-ray-radiation-computer-glasses-270591685.html
Scraping reviews for: https://www.jumia.com.ng/fashion-couple-ring-for-love-256454120.html
Scraping reviews for: https://www.jumia.com.ng/ao-eyewear-black-military-luxury-designer-fashion-pilot-sunglasses-63893226.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-sleeveless-casual-top-270554311.html
Scraping reviews for: https://www.jumia.com.ng/fuce-unisex-casual-school-sandal-black-165299086.html
Scraping reviews for: https://www.jumia.com.ng/fashion-4-pcs-women-bags-ladies-bags-handbags-purse-shoulder-bags-red-wine-99251225.html
Scraping reviews for: https://www.jumia.com.ng/fashion-dazzling-very-stylish-silver-wristwatch-double-romantic-wrist-set-for-classic-ladies-135850527.html
Scraping reviews for: https://www.jumia.com.ng/geneva-unisex-luxury-wristwatch-with-night-light-63474575.html
Scraping reviews for: https://www.jumia.com.ng/fashion-multi-timer-clas

Scraping reviews for: https://www.jumia.com.ng/keep-moving-i-am-king-iced-top-stylish-collection-of-wristwatch-iced-necklacering-bracelet-for-womenmen-140909750.html
Scraping reviews for: https://www.jumia.com.ng/fashion-19-big-star.simple-earrings-round-face-thin-temperament-retro-fashion-personality-earrings-226359483.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-hand-bag-tote-bag-pu-leather-shoulder-bag-chain-crossbody-bag-for-women-135501761.html
Scraping reviews for: https://www.jumia.com.ng/adidas-tensaur-hook-and-loop-shoes-unisex-272135040.html
Scraping reviews for: https://www.jumia.com.ng/generic-womens-sneakers-air-cushion-breathable-non-slip-leisure-black-274010214.html
Scraping reviews for: https://www.jumia.com.ng/fashion-quality-unisex-plain-black-face-cap-108604317.html
Scraping reviews for: https://www.jumia.com.ng/fashion-cuban-link-chain-with-star-of-david-pendant-gold-71785963.html
Scraping reviews for: https://www.jumia.com.ng/fashion-iner-cott

Scraping reviews for: https://www.jumia.com.ng/xcst-bnzho-men-sandals-summer-beach-holiday-sandals-men-shoes-new-outdoor-male-comfortable-casual-sandals-men-sneakers-large-size-38-47-129456333.html
Scraping reviews for: https://www.jumia.com.ng/adidas-originals-sneakers-stan-smith-86787515.html
Scraping reviews for: https://www.jumia.com.ng/generic-portable-lcd-digital-hand-travel-luggage-scale-40kg-261372252.html
Scraping reviews for: https://www.jumia.com.ng/fashion-elegant-women-rhinestone-wrist-watch-and-bracelet-gold-52448588.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-shoes-trend-sports-sneakers-blackwhite-217425490.html
Scraping reviews for: https://www.jumia.com.ng/fashion-4-pcs-women-bags-ladies-bags-handbags-purse-shoulder-bags-273691815.html
Scraping reviews for: https://www.jumia.com.ng/fashion-trendy-mens-and-women-wristwatch-combo-with-bracelet-luxury-83212391.html
Scraping reviews for: https://www.jumia.com.ng/fashion-arrowhead-pendant-men-necklace-v

Scraping reviews for: https://www.jumia.com.ng/two-pieces-quality-chinos-for-men-black-carton-color-fashion-mpg3889889.html
Scraping reviews for: https://www.jumia.com.ng/adidas-sports-performance-socks-light-nosh-3pp-73825518.html
Scraping reviews for: https://www.jumia.com.ng/fashion-automatic-buckle-genuine-leather-belt-black-76195854.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-shoes-2022-mens-casual-shoes-breathable-running-sneakers-98297394.html
Scraping reviews for: https://www.jumia.com.ng/fashion-original-quality-iced-out-cuban-tennis-link-steel-chain-with-cross-diamond-pendant-104864909.html
Scraping reviews for: https://www.jumia.com.ng/magnus-paige-mens-casual-genuine-pam-slippers-fashion-mpg2182368.html
Scraping reviews for: https://www.jumia.com.ng/skmei-elegance-top-luxury-designer-digital-wristwatch-83745981.html
Scraping reviews for: https://www.jumia.com.ng/nepic-limited-edition-executive-dutch-chrono-by-nel-picter-70471923.html
Scraping reviews fo